# Solar Analysis

### Import, read, display:

In [45]:
# solar data
# fill in 0s for sq_ft, hrs_available, since impt to see if solar viable option
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math


In [55]:
solar_data = pd.read_csv('solar_data.csv')
census_data = pd.read_csv('census_data.csv')
gas_data = pd.read_csv('gas_meter_data.csv')
total_gas_data = pd.read_csv('total_gas_data.csv')

In [ ]:
print('HEAD', solar_data.head())
#other things

HEAD    house_id  device_id  gas_meter_id        lat       long  \
0  b1eb5eb2   83771758           NaN  42.191697 -72.613522   
1  b1eb5eb2   69525019           NaN  42.191697 -72.613522   
2  a90ea0b3    5827516    54363558.0  42.211167 -72.621543   
3  049bf03c   83770216    54459969.0  42.211157 -72.621317   
4  b2dc9fbe   56976532           NaN  42.210828 -72.621242   

   hours_of_sunlight_per_year  sq_ft_available_for_solar_panels  
0                        1404                              3207  
1                           0                                 0  
2                        1488                              1180  
3                        1504                              1163  
4                        1383                               846  


In [ ]:
print('INFO', solar_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17052 entries, 0 to 17051
Data columns (total 7 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   house_id                          17052 non-null  object 
 1   device_id                         17052 non-null  int64  
 2   gas_meter_id                      10162 non-null  float64
 3   lat                               17052 non-null  float64
 4   long                              17052 non-null  float64
 5   hours_of_sunlight_per_year        17052 non-null  int64  
 6   sq_ft_available_for_solar_panels  17052 non-null  int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 932.7+ KB
INFO None


In [ ]:
print('DESC', solar_data.describe())

DESC           device_id  gas_meter_id           lat          long  \
count  1.705200e+04  1.016200e+04  17052.000000  17052.000000   
mean   4.486838e+07  5.648602e+07     42.202733    -72.623131   
std    4.449656e+07  1.977618e+07      0.069791      0.545868   
min    5.819832e+06  2.029568e+07     40.771841    -73.501843   
25%    2.039324e+07  4.626168e+07     42.194981    -72.635908   
50%    3.105131e+07  4.976170e+07     42.203596    -72.624741   
75%    5.676430e+07  5.635645e+07     42.210083    -72.615631   
max    3.383795e+08  1.007938e+08     50.929574     -1.391057   

       hours_of_sunlight_per_year  sq_ft_available_for_solar_panels  
count                17052.000000                      17052.000000  
mean                   730.968156                        885.132301  
std                    685.065756                       3563.142914  
min                      0.000000                          0.000000  
25%                      0.000000                          

### KNN Imputation:

#### 1. Data processing:

In [56]:
# remove independent variable columns
solar_knn = solar_data.drop(['device_id', 'gas_meter_id'], axis=1)

# detect missing values
print('missing:') 
print(solar_knn.eq(0).sum(), '\n')

#how many not missing
print('not missing:') 
print(solar_knn.ne(0).sum())

missing:
house_id                               0
lat                                    0
long                                   0
hours_of_sunlight_per_year          7910
sq_ft_available_for_solar_panels    9124
dtype: int64 

not missing:
house_id                            17052
lat                                 17052
long                                17052
hours_of_sunlight_per_year           9142
sq_ft_available_for_solar_panels     7928
dtype: int64


In [91]:
gas_house_ids = set(gas_data['house_id'])
solar_house_ids = set(solar_data['house_id'])

common_house_ids = gas_house_ids.intersection(solar_house_ids)
print(f"Number of common house_ids: {len(common_house_ids)}")
#should be able to do knn with gas usage on these houses, yay


Number of common house_ids: 5747


In [92]:
solar_knn = pd.merge(solar_data, total_gas_data, on='house_id', how='left')
solar_knn.head()
print(solar_knn['total_gas_usage'].isna().sum())  # Count how many NaNs

12663


In [58]:
blocks_and_house_ids = census_data[['Block', 'house_ids']]
solar_house_ids = solar_data['house_id']

house_id_to_block = {}
for solar_row in range(len(solar_data.index)):
    for census_row in range(len(census_data.index)):
        house_ids_as_list = blocks_and_house_ids['house_ids'].loc[census_row]
        solar_house_id = solar_house_ids.loc[solar_row]
        if solar_house_id in house_ids_as_list:
            house_id_to_block[solar_house_id] = blocks_and_house_ids['Block'].loc[census_row]

solar_knn['Block'] = solar_knn['house_id'].map(house_id_to_block)


In [59]:
# detect missing values
print('missing:') 
print(solar_knn.eq(0).sum(), '\n')

#how many not missing
print('not missing:') 
print(solar_knn.ne(0).sum())

missing:
house_id                               0
device_id                              0
gas_meter_id                           0
lat                                    0
long                                   0
hours_of_sunlight_per_year          7910
sq_ft_available_for_solar_panels    9124
total_gas_usage                        0
Block                                  0
dtype: int64 

not missing:
house_id                            17052
device_id                           17052
gas_meter_id                        17052
lat                                 17052
long                                17052
hours_of_sunlight_per_year           9142
sq_ft_available_for_solar_panels     7928
total_gas_usage                     17052
Block                               17052
dtype: int64


In [93]:
#remove duplicate house_ids
solar_data = solar_data.drop_duplicates(subset='house_id')
total_gas_data = total_gas_data.drop_duplicates(subset='house_id')

#standardize
solar_data['house_id'] = solar_data['house_id'].str.strip().str.lower()
total_gas_data['house_id'] = total_gas_data['house_id'].str.strip().str.lower()

In [94]:
solar_gas_data = pd.merge(solar_data, total_gas_data, on='house_id', how='left')


,house_id,device_id,gas_meter_id,lat,long,hours_of_sunlight_per_year,sq_ft_available_for_solar_panels,total_gas_usage
0,b1eb5eb2,83771758,NaN,42.191697,-72.613522,1404,3207,NaN
1,a90ea0b3,5827516,54363558.0,42.211167,-72.621543,1488,1180,NaN
2,049bf03c,83770216,54459969.0,42.211157,-72.621317,1504,1163,NaN
3,b2dc9fbe,56976532,NaN,42.210828,-72.621242,1383,846,NaN
4,569aaf4b,56506585,40488619.0,42.210727,-72.621080,1337,440,21377.52


In [96]:
solar_gas_data = solar_gas_data[solar_gas_data['total_gas_usage'].notna() & (solar_gas_data['total_gas_usage'] != 0)]
print(len(solar_gas_data))

4352


In [83]:
# only want to do knn on houses we care about (those that use gas) to save time
# find gas houses w/ at least 1 missing value

knn_houses = solar_knn[(solar_knn['hours_of_sunlight_per_year'] == 0) |(solar_knn['sq_ft_available_for_solar_panels'] == 0)]
print(len(knn_houses))

9124


In [85]:
final_knn_df = knn_houses[['house_id', 'Block', 'total_gas_usage', 'hours_of_sunlight_per_year', 'sq_ft_available_for_solar_panels']]

print(final_knn_df.head())

    house_id     Block  total_gas_usage  hours_of_sunlight_per_year  \
1   b1eb5eb2  812103-1              NaN                           0   
6   ac2f7c77  812001-3              NaN                        1359   
7   c9a15592  811900-1              NaN                        1205   
9   ae254799  811900-1              NaN                        1205   
10  1ad80ffe  811900-1              NaN                        1205   

    sq_ft_available_for_solar_panels  
1                                  0  
6                                  0  
7                                  0  
9                                  0  
10                                 0  


In [68]:
# proceeding with KNN, total gas usage as numeric variable and block as categorical

#one-hot encoding for block
final_knn_df = pd.get_dummies(final_knn_df, columns=['Block'], prefix='Block')

In [69]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
final_knn_df['total_gas_usage'] = scaler.fit_transform(final_knn_df[['total_gas_usage']])


In [76]:
print(len(final_knn_df))



37


In [71]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5, weights='distance')

columns_for_imputation = ['total_gas_usage'] + [col for col in final_knn_df.columns if 'Block' in col] + ['hours_of_sunlight_per_year', 'sq_ft_available_for_solar_panels']
final_knn_df_imputed = imputer.fit_transform(final_knn_df[columns_for_imputation])

final_knn = pd.DataFrame(final_knn_df_imputed, columns=columns_for_imputation)


total_gas_usage                     0
Block_811400-1                      0
Block_811400-2                      0
Block_811500-1                      0
Block_811500-2                      0
Block_811600-1                      0
Block_811600-2                      0
Block_811600-3                      0
Block_811800-3                      0
Block_811800-5                      0
Block_811900-2                      0
Block_811900-3                      0
Block_812002-1                      0
Block_812002-4                      0
Block_812101-1                      0
Block_812101-2                      0
Block_812101-3                      0
Block_812103-1                      0
Block_812103-2                      0
Block_812104-3                      0
Block_812104-4                      0
Block_822500-4                      0
hours_of_sunlight_per_year          0
sq_ft_available_for_solar_panels    0
dtype: int64


In [75]:
print(len(final_knn))

37
